In [8]:
import json
import os
import numpy as np
import pandas as pdf
from tqdm import tqdm_notebook as tqdm

# Examine Zeerak NAACL tweets

In [58]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)

In [62]:
datapath = '/usr0/home/mamille2/11-830_data/project/zeerak_naacl/zeerak_naacl_tweets.pkl'
data = pd.read_pickle(datapath)
data.loc[:, ['text', 'label']]

,text,label
0,"@ummayman90 @wood_brwood333 For example, in Medina Mohammed said, kindly cut off the heads of these 600 Jewish men and enslave their women.",racism
1,"@ummayman90 Again, your entire concept of god corresponds to a tyrannical earthly egomaniac because you are simple and stupid. #Islam",racism
2,"@anjemchoudary Your prophet was a rapist, murderer, pedophile, caravan robber, slave trader, bigot and sexist. God would never use the scum.",racism
3,"RT @DilanaKurdi: Yazidi children who are taken from their parents, forcibly converted to Islam! #Sinjar #Shengal http://t.co/wwzplHld7f",racism
4,@anjemchoudary Accepting the idea that Allah obsesses over how we take a dump shows just how utterly insane this inbred religion is.,racism
5,"These girls are the equivalent of the irritating Asian girls a couple years ago. Well done, 7. #MKR",racism
6,@ummayman90 Who knows if they criticized him? He killed them. And the things against him would never make it into the Sunnah.,racism
7,@lauracdean I love how the Islamofascists recruit 14 and 15 year old jihadis and then talk about minors in reference to 17 year olds.,racism
8,RT @PeterTownsend7: #ISIS #Islam PC Puzzle: Converting to a religion of peace leading to violence? http://t.co/TBJUsaeMUH http://t.co/G4xOh…,racism
9,@yousufpoosuf @RKingLive2Dance @izrinhariri And most Muslims are actually better behaved than their religion leads them to be.,racism


In [63]:
len(data[data['label']=='none'])

10853

In [60]:
data.loc[:, 'text', 'label', 'tags']

IndexingError: Too many indexers

# Assemble tweepy-downloaded tweets

In [53]:
data_dirpath = '/usr0/home/mamille2/11-830_data/project/zeerak_naacl/'
out_pklpath = '/usr0/home/mamille2/11-830_data/project/zeerak_naacl/zeerak_naacl_tweets.pkl'
out_csvpath = '/usr0/home/mamille2/11-830_data/project/zeerak_naacl/zeerak_naacl_tweets.csv'

file_lens = []
outlines = []

for fname in tqdm(sorted(os.listdir(data_dirpath))):
    with open(os.path.join(data_dirpath, fname)) as f:
        tweets = json.load(f)
        file_lens.append(len(tweets))

        for t in tweets:

            # Extract info from tweet
            info = [t['id'], t['created_at'], t['in_reply_to_status_id'], t['lang'], t['retweet_count'], 
                    t['user']['id'], t['user']['screen_name'], t['user']['name'],
                    t['text'], t['entities']['hashtags']]

            outlines.append(info)

print(np.mean(file_lens))

out = pd.DataFrame(outlines, columns=['tweet_id', 'created_at', 'in_reply_to_status_id', 'lang',
                                 'retweet_count', 'user_id', 'user_screen_name', 'user_name',
                                 'text', 'tags'])

print(len(out))
out.to_pickle(out_pklpath)
out.to_csv(out_csvpath, index=False)

In [16]:
# Get tweet labels
labeled_data = pd.read_csv('/usr0/home/mamille2/11-830_data/project/zeerak_naacl/NAACL_SRW_2016.csv', header=None,
                          names=['tweet_id', 'label'])
len(labeled_data)

16907

In [42]:
merged = pd.merge(out, labeled_data, how='right')
print(merged.columns)
print(len(merged))

Index(['tweet_id', 'created_at', 'in_reply_to_status_id', 'lang',
       'retweet_count', 'user_id', 'user_screen_name', 'user_name', 'text',
       'tags', 'label'],
      dtype='object')
17007


In [23]:
len(out)

15913

In [24]:
len(merged['tweet_id'].unique())

15863

In [43]:
# Drop duplicates
merged.drop_duplicates(subset=['tweet_id'], inplace=True, keep=False)
len(merged)

16791

In [49]:
# Drop label and text NaN rows
merged.dropna(axis=0, subset=['label', 'text'], inplace=True)
len(merged)

15813

In [61]:
merged.to_pickle(out_pklpath)
merged.to_csv(out_csvpath, index=False)

In [34]:
merged['text']

0        @ummayman90 @wood_brwood333 For example, in Me...
1        @ummayman90 Again, your entire concept of god ...
2        @anjemchoudary Your prophet was a rapist, murd...
3        RT @DilanaKurdi: Yazidi children who are taken...
4        @anjemchoudary Accepting the idea that Allah o...
5        These girls are the equivalent of the irritati...
6        @ummayman90 Who knows if they criticized him? ...
7        @lauracdean I love how the Islamofascists recr...
8        RT @PeterTownsend7: #ISIS #Islam PC Puzzle: Co...
9        @yousufpoosuf @RKingLive2Dance @izrinhariri An...
14       @HillaryGuess @riwired @izrinhariri @AtharHKha...
15       @ummayman90 Because they breed like rats, kill...
16       @ummayman90 Again, you miss the point that god...
17       RT @dsyndergaard: Israeli spat upon, beaten by...
18       @TII99 That is a total lie.  Minorities in Mus...
19       It seems that Allah sits around all day obsess...
20       @yousufpoosuf @RKingLive2Dance @izrinhariri Th.

In [33]:
gped = out.groupby('tweet_id').size()
dups = gped[gped > 1]
dups

tweet_id
552487055553757187    2
552923665952546816    2
572330982029709312    2
572331686156873728    2
572335493586685953    2
572336047775870976    2
572336241871478786    2
572336280186429440    2
572336293725642752    2
572336406053322753    2
572336427867885568    2
572336451221757953    2
572336657384407040    2
572336799340630016    2
572336970979885056    2
572337067436351489    2
572337337318817792    2
572337509159452672    2
572337676482838528    2
572337676713512960    2
572338147700293632    2
572338419851911168    2
572338617772740608    2
572338619282694144    2
572338780671107072    2
572339081780326400    2
572339216178270208    2
572339607829794816    2
572339639240929281    2
572340250715930624    2
572340476503724032    2
572340883489615872    2
572341118118936577    2
572341307584073728    2
572341498827522049    2
572342652907044864    2
572342944256032768    2
572342978255048705    2
572343206479712256    2
572343828968943616    2
572344263389794304    2
5723449

In [29]:
merged[merged['tweet_id'].isin(dups.index)]

,tweet_id,created_at,in_reply_to_status_id,lang,retweet_count,user_id,user_screen_name,user_name,text,tags,label
10,572341498827522049,Mon Mar 02 10:23:41 +0000 2015,NaN,en,0,110114783,trish2295,patricia hilder,Drasko they didn't cook half a bird you idiot ...,"[{'text': 'mkr', 'indices': [46, 50]}]",racism
11,572341498827522049,Mon Mar 02 10:23:41 +0000 2015,NaN,en,0,110114783,trish2295,patricia hilder,Drasko they didn't cook half a bird you idiot ...,"[{'text': 'mkr', 'indices': [46, 50]}]",none
12,572341498827522049,Mon Mar 02 10:23:41 +0000 2015,NaN,en,0,110114783,trish2295,patricia hilder,Drasko they didn't cook half a bird you idiot ...,"[{'text': 'mkr', 'indices': [46, 50]}]",racism
13,572341498827522049,Mon Mar 02 10:23:41 +0000 2015,NaN,en,0,110114783,trish2295,patricia hilder,Drasko they didn't cook half a bird you idiot ...,"[{'text': 'mkr', 'indices': [46, 50]}]",none
22,572340476503724032,Mon Mar 02 10:19:37 +0000 2015,NaN,en,0,38650214,foodbling,"food bling, Brisbane",Hopefully someone cooks Drasko in the next ep ...,"[{'text': 'MKR', 'indices': [49, 53]}]",racism
23,572340476503724032,Mon Mar 02 10:19:37 +0000 2015,NaN,en,0,38650214,foodbling,"food bling, Brisbane",Hopefully someone cooks Drasko in the next ep ...,"[{'text': 'MKR', 'indices': [49, 53]}]",none
24,572340476503724032,Mon Mar 02 10:19:37 +0000 2015,NaN,en,0,38650214,foodbling,"food bling, Brisbane",Hopefully someone cooks Drasko in the next ep ...,"[{'text': 'MKR', 'indices': [49, 53]}]",racism
25,572340476503724032,Mon Mar 02 10:19:37 +0000 2015,NaN,en,0,38650214,foodbling,"food bling, Brisbane",Hopefully someone cooks Drasko in the next ep ...,"[{'text': 'MKR', 'indices': [49, 53]}]",none
43,552487055553757187,Tue Jan 06 15:29:13 +0000 2015,NaN,en,2,2941145694,VileIslam,Levi Stein,RT @Benfrancisallen: It hasn't been a good few...,"[{'text': 'ISIS', 'indices': [57, 62]}, {'text...",racism
44,552487055553757187,Tue Jan 06 15:29:13 +0000 2015,NaN,en,2,2941145694,VileIslam,Levi Stein,RT @Benfrancisallen: It hasn't been a good few...,"[{'text': 'ISIS', 'indices': [57, 62]}, {'text...",none


In [30]:
len(labeled_data['tweet_id'].unique())

16849